# Assignment 1: Dino Fun World

You, in your role as a burgeoning data explorer and visualizer, have been asked by the administrators of a small amusement park in your hometown to answer a couple questions about their park operations. In order to perform the requested analysis, they have provided you with a database containing information about one day of the park's operations.

### Provided Database

The database provided by the park administration is formatted to be readable by any SQL database library. The course staff recommends the sqlite3 library. The database contains three tables, named 'checkins', 'attractions', and 'sequences'. The information contained in each of these tables is listed below:

`checkins`:
    - Description: check-in data for all visitors for the day in the park. The data includes two types of check-ins, inferred and actual checkins.
    - Fields: visitorID, timestamp, attraction, duration, type
`attraction`:
    - The attractions in the park by their corresponding AttractionID, Name, Region, Category, and type. Regions are from the VAST Challenge map such as Coaster Alley, Tundra Land, etc. Categories include Thrill rides, Kiddie Rides, etc. Type is broken into Outdoor Coaster, Other Ride, Carussel, etc.
    - Fields: AttractionID, Name, Region, Category, type
`sequences`:
    - The check-in sequences of visitors. These sequences list the position of each visitor to the park every five minutes. If the visitor has not entered the part yet, the sequence has a value of 0 for that time interval. If the visitor is in the park, the sequence lists the attraction they have most recently checked in to until they check in to a new one or leave the park.
    - Fields: visitorID, sequence
    
The database is named 'dinofunworld.db' and is located in the 'readonly' directory of the Jupyter Notebook environment. It can be accessed at 'readonly/dinofunworld.db'.

### Questions to Answer

The administrators would like you to answer four relatively simple questions about the park activities on the day in question. These questions all deal with park operations and can be answered using the data provided.

Question 1: What is the most popular attraction to visit in the park?
Question 2: What ride (note that not all attractions are rides) has the longest visit time?
Question 3: Which Fast Food offering has the fewest visitors?
Question 4: Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline.

#### Administrative Notes

This assignment will be graded by Coursera's grading system. In order for your answers to be correctly registered in the system, you must place the code for your answers in the cell indicated for each question. In addition, you should submit the assignment with the output of the code in the cell's display area. The display area should contain only your answer to the question with no extraneous information, or else the answer may not be picked up correctly. Each cell that is going to be graded has a set of comment lines at the beginning of the cell. These lines are extremely important and must not be modified or removed.

In [1]:
import sqlite3
from datetime import timedelta
import math
con = sqlite3.connect('readonly/dinofunworld.db')
cur = con.cursor()

In [2]:
# Graded Cell
# PartID:  NDnou
# Question 1: What is the most popular attraction to visit in the park?
# Notes: Your output should be the name of the attraction.
cur.execute("SELECT attraction, COUNT(*) as c FROM checkin GROUP BY attraction ORDER BY c desc;")
popular_attraction_id = cur.fetchall()[0][0]
cur.execute("SELECT Name FROM attraction WHERE AttractionID="+str(popular_attraction_id)+";")
popular_attraction = cur.fetchone()[0]
print(popular_attraction)

Atmosfear


In [3]:
# Graded Cell
# PartID: FXGHp
# Question 2: What ride (note that not all attractions are rides) has the longest average visit time?
# Notes: Your output should be the name of the ride.
cur.execute("SELECT AttractionID, Name FROM attraction where LOWER(Category) LIKE '%ride%';")
attraction_name_category_list = cur.fetchall()
ride_time_tuple = ("", 0)
for (aid, name) in attraction_name_category_list:
    cur.execute("SELECT duration FROM checkin where attraction="+str(aid)+";")
    duration_list = cur.fetchall()
    total_dur = 0
    n = 0
    for duration in duration_list:
        try:
            dur = duration[0].split(':')
            total_dur += timedelta(int(dur[0]), int(dur[1]), int(dur[2])).total_seconds()
            n += 1
        except:
            continue
    avg_dur = total_dur / n
    if avg_dur > ride_time_tuple[1]:
        ride_time_tuple = (name, avg_dur)
print(ride_time_tuple[0])

Flight of the Swingodon


In [4]:
# Graded Cell
# PartID: KALua
# Question 3: Which Fast Food offering in the park has the fewest visitors?
# Notes: Your output should be the name of the fast food offering.
cur.execute("SELECT attraction, COUNT(*) as c FROM checkin WHERE attraction IN (SELECT AttractionID FROM attraction where LOWER(type) LIKE '%fast food%') GROUP BY attraction ORDER by c asc;")
attraction_id = cur.fetchall()[0][0]
cur.execute("SELECT Name FROM attraction where AttractionID = " + str(attraction_id) + ";")
attraction_name = cur.fetchone()[0]
print(attraction_name)

Theresaur Food Stop


In [5]:
# Graded Cell
# PartID: B0LUP
# Question 4: Compute the Skyline of number of visits and visit time for the park's ride and 
#  report the rides that appear in the Skyline. 
# Notes: Remember that in this case, higher visits is better and lower visit times are better. 
#  Your output should be formatted as an array listing the names of the rides in the Skyline.
cur.execute("SELECT attraction, COUNT(*) as c FROM checkin WHERE attraction IN (SELECT AttractionID FROM attraction where LOWER(Category) LIKE '%ride%') GROUP BY attraction;")
visit_list = cur.fetchall()
best_visited_tuple = (0, 0)
best_ride_time_tuple = (0, math.inf)
best_both_tuple = (0, 0, math.inf)
for (aid,count) in visit_list:
    cur.execute("SELECT duration FROM checkin where attraction="+str(aid)+";")
    duration_list = cur.fetchall()
    total_dur = 0
    n = 0
    for duration in duration_list:
        try:
            dur = duration[0].split(':')
            total_dur += timedelta(int(dur[0]), int(dur[1]), int(dur[2])).total_seconds()
            n += 1
        except:
            continue
    avg_dur = total_dur / n
    if count > best_visited_tuple[1]:
        best_visited_tuple = (aid, count)
    if avg_dur < best_ride_time_tuple[1]:
        best_ride_time_tuple = (aid, avg_dur)
    if count > best_both_tuple[1] and avg_dur < best_both_tuple[2]:
        best_both_tuple = (aid, count, avg_dur)
cur.execute("SELECT Name FROM attraction WHERE AttractionID="+str(best_visited_tuple[0])+" OR AttractionID="+str(best_ride_time_tuple[0])+" OR AttractionID="+str(best_both_tuple[0])+";")
names = cur.fetchall()
name_list = [name[0] for name in names]
print(name_list)

['Wrightiraptor Mountain', 'Atmosfear', 'Dykesadactyl Thrill']
